In [9]:
import sys
import pandas as pd
import yfinance as yf
from datetime import datetime, time, timedelta

In [10]:
def get_open_close_prices(symbol, timestamp):
    try:
        print("symbol: ", symbol)
        print("timestamp: ", timestamp)

        date_obj = datetime.strptime(timestamp, "%a %b %d %H:%M:%S +0000 %Y")
        print(date_obj)

        market_closing_time = time(16, 0, 0)

        if date_obj.time() <= market_closing_time:
            start_date = datetime(date_obj.year, date_obj.month, date_obj.day, 0, 0, 0)
            end_date = datetime(date_obj.year, date_obj.month, date_obj.day, 16, 0, 0)
            print("start_date same day: ", start_date)
            print("end_date same day: ", end_date)
        else:
            start_date = datetime(date_obj.year, date_obj.month, date_obj.day, 16, 0, 0)
            end_date = start_date + timedelta(days=1)
            print("start_date next day: ", start_date)
            print("end_date next day: ", end_date)

        data = yf.download(symbol, start=start_date, end=end_date)
        print(data)
        if data.empty:
            open_price, close_price = None, None
        else:
            open_price = data['Open'].iloc[0]
            close_price = data['Close'].iloc[0]

    except Exception as e:
        open_price, close_price = None, None

    return open_price, close_price

In [11]:
chunk_size = 20
output_file = '/StockMarketCommunityDetector/result/output_data.csv'

In [12]:
with pd.read_csv('/StockMarketCommunityDetector/dataset/stockerbot-export.csv', usecols=['timestamp', 'source', 'symbols', 'company_names'], chunksize=chunk_size) as reader:
    for chunk in reader:
        open_prices = []
        close_prices = []

        for index, row in chunk.iterrows():
            symbol = row['symbols'].strip()
            timestamp = row['timestamp']

            open_price, close_price = get_open_close_prices(symbol, timestamp)

            open_prices.append(open_price)
            close_prices.append(close_price)


        chunk['open'] = open_prices
        chunk['close'] = close_prices


        chunk.to_csv(output_file, mode='a', header=False, index=False)

symbol:  GS
timestamp:  Wed Jul 18 21:33:26 +0000 2018
2018-07-18 21:33:26
start_date next day:  2018-07-18 16:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2018-07-18  232.339996  233.720001  230.149994  231.240005  205.497879   
2018-07-19  231.110001  231.990005  228.979996  229.630005  204.067108   

             Volume  
Date                 
2018-07-18  6149900  
2018-07-19  4638500  
symbol:  M
timestamp:  Wed Jul 18 22:22:47 +0000 2018
2018-07-18 22:22:47
start_date next day:  2018-07-18 16:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2018-07-18  37


1 Failed download:
['BTC']: Exception("%ticker%: Data doesn't exist for startDate = 1531944000, endDate = 1532030400")



Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
symbol:  ORCL
timestamp:  Wed Jul 18 23:00:01 +0000 2018
2018-07-18 23:00:01
start_date next day:  2018-07-18 16:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2018-07-18  48.790001  49.000000  48.400002  48.639999  44.634518  19234700
2018-07-19  48.400002  48.720001  48.209999  48.439999  44.450981  16656500
symbol:  FB-GOOGL-GOOG
timestamp:  Wed Jul 18 23:04:00 +0000 2018
2018-07-18 23:04:00
start_date next day:  2018-07-18 16:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['FB-GOOGL-GOOG']: Exception('%ticker%: No timezone found, symbol may be delisted')



Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
symbol:  HRS
timestamp:  Wed Jul 18 23:06:58 +0000 2018
2018-07-18 23:06:58
start_date next day:  2018-07-18 16:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed

...


1 Failed download:
['TMK']: Exception('%ticker%: No timezone found, symbol may be delisted')



Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
symbol:  ORLY
timestamp:  Tue Jul 10 03:03:25 +0000 2018
2018-07-10 03:03:25
start_date same day:  2018-07-10 00:00:00
end_date same day:  2018-07-10 16:00:00
[*********************100%%**********************]  1 of 1 completed
                  Open       High         Low       Close   Adj Close  Volume
Date                                                                         
2018-07-10  289.399994  290.01001  285.850006  288.109985  288.109985  488800
symbol:  TMK
timestamp:  Tue Jul 10 03:04:31 +0000 2018
2018-07-10 03:04:31
start_date same day:  2018-07-10 00:00:00
end_date same day:  2018-07-10 16:00:00
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['TMK']: Exception('%ticker%: No timezone found, symbol may be delisted')



Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
symbol:  IP
timestamp:  Tue Jul 10 03:18:38 +0000 2018
2018-07-10 03:18:38
start_date same day:  2018-07-10 00:00:00
end_date same day:  2018-07-10 16:00:00
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2018-07-10  50.265152  50.710228  50.265152  50.634472  39.889675  1972080
symbol:  IP
timestamp:  Tue Jul 10 03:19:31 +0000 2018
2018-07-10 03:19:31
start_date same day:  2018-07-10 00:00:00
end_date same day:  2018-07-10 16:00:00
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2018-07-10  50.265152  50.710228  50.265152  50.634472   39.88969  1972080
symbol:  IP
timest

KeyboardInterrupt: 